In [6]:
import sqlite3

# Assuming you have a DataFrame named 'df' with columns for fixtures and odds for different bets/markets.
# To display all unique fixtures and their corresponding odds for each market:

# Display all rows to see all possible odds for each fixture and market
pd.set_option('display.max_rows', None)

conn = sqlite3.connect('db_sportmonks.db')
cursor = conn.cursor()

# Print all tables in the database
print("Tables in the database:")
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
for table in tables:
    print(table[0])

# Close the connection
conn.close()

Tables in the database:
continents
countries
regions
cities
leagues
seasons
schedules
stages
rounds
teams
players
coaches
referees
squads
sqlite_sequence
fixtures
events
lineups
commentaries
standings
statistics
top_scorers
bookmakers
markets
odds
trends
venues
tv_stations
groups
transfers
rivals
news
expected_xg
team_squads
topscorers
penalties
cards
processed_fixtures
checkpoint
fixture_odds
processed_fixtures_odds
odds_checkpoint
fixture_team_names
team_name_mapping
odds_cache
evaluation
predictions


In [16]:
# Step 1: Explore the database schema and sample data for understanding

# List all tables in the database (already done in previous cell)
print("Tables in the database:")
for table in tables:
    print(table[0])

# Let's inspect the schema of key tables: 'odds', 'predictions', and 'fixtures'
def print_table_schema(cursor, table_name):
    print(f"\nSchema for table '{table_name}':")
    schema = cursor.execute(f"PRAGMA table_info({table_name});").fetchall()
    for col in schema:
        print(col)

key_tables = ['odds', 'predictions', 'fixtures']
for t in key_tables:
    print_table_schema(cursor, t)

# Preview a few rows from each key table to understand the data
def preview_table(cursor, table_name, n=5):
    print(f"\nSample rows from '{table_name}':")
    rows = cursor.execute(f"SELECT * FROM {table_name} LIMIT {n};").fetchall()
    for row in rows:
        print(row)

for t in key_tables:
    preview_table(cursor, t)

# After this, we will have a clear understanding of the structure and content of the main tables.
# This will guide us in building the data pipeline for training, predicting, updating, and deploying.

Tables in the database:
continents
countries
regions
cities
leagues
seasons
schedules
stages
rounds
teams
players
coaches
referees
squads
sqlite_sequence
fixtures
events
lineups
commentaries
standings
statistics
top_scorers
bookmakers
markets
odds
trends
venues
tv_stations
groups
transfers
rivals
news
expected_xg
team_squads
topscorers
penalties
cards
processed_fixtures
checkpoint
fixture_odds
processed_fixtures_odds
odds_checkpoint
fixture_team_names
team_name_mapping
odds_cache
evaluation
predictions

Schema for table 'odds':
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'fixture_id', 'INTEGER', 0, None, 0)
(2, 'bookmaker_id', 'INTEGER', 0, None, 0)
(3, 'market_id', 'INTEGER', 0, None, 0)
(4, 'label', 'TEXT', 0, None, 0)
(5, 'odd_value', 'REAL', 0, None, 0)
(6, 'updated_at', 'TEXT', 0, None, 0)
(7, 'raw', 'TEXT', 0, None, 0)

Schema for table 'predictions':
(0, 'fixture_id', 'BIGINT', 0, None, 1)
(1, 'league', 'TEXT', 0, None, 0)
(2, 'kickoff', 'TIMESTAMPTZ', 0, None, 0)
(3, 'home_team', 'TEX

In [17]:
# Load all rows from the 'odds' table into a DataFrame
df_odds = pd.read_sql_query("SELECT * FROM odds", conn)
df_odds.head()

,id,fixture_id,bookmaker_id,market_id,label,odd_value,updated_at,raw
0,117314740794,19138512,38,1,Home,6.05,2025-05-11T11:11:27.929085,"{""id"": 117314740794, ""fixture_id"": 19138512, ""..."
1,117314740795,19138512,38,1,Draw,4.00,2025-05-11T11:11:27.929119,"{""id"": 117314740795, ""fixture_id"": 19138512, ""..."
2,117314740796,19138512,38,1,Away,1.59,2025-05-11T11:11:27.929096,"{""id"": 117314740796, ""fixture_id"": 19138512, ""..."
3,117315706427,19138512,38,6,Home,1.85,2025-05-11T11:11:27.936061,"{""id"": 117315706427, ""fixture_id"": 19138512, ""..."
4,117315706428,19138512,38,6,Away,1.79,2025-05-11T11:11:27.936070,"{""id"": 117315706428, ""fixture_id"": 19138512, ""..."


In [18]:
# Check the structure and sample values of odds_sample to ensure odds data is correct
for row in odds_sample:
    print(row)

# If you need to fetch more odds or update how they are stored, you can query the database as needed.
# Example: Fetch all odds for a specific fixture_id (replace 19138512 with your desired fixture_id)
fixture_id_to_check = 19138512
conn = sqlite3.connect('db_sportmonks.db')
df_specific_odds = pd.read_sql_query(
    f"SELECT * FROM odds WHERE fixture_id = {fixture_id_to_check}", conn
)
conn.close()

# Display the fetched odds
df_specific_odds.head()

(117314740794, 19138512, 38, 1, 'Home', 6.05, '2025-05-11T11:11:27.929085', '{"id": 117314740794, "fixture_id": 19138512, "market_id": 1, "bookmaker_id": 38, "label": "Home", "value": "6.05", "name": "Home", "sort_order": 1, "market_description": "Fulltime Result", "probability": "16.52%", "dp3": "6.055", "fractional": "103/17", "american": "505", "winning": false, "stopped": false, "total": null, "handicap": null, "participants": null, "created_at": "2024-08-04T20:46:44.000000Z", "original_label": null, "latest_bookmaker_update": "2025-05-04 10:48:36"}')
(117314740795, 19138512, 38, 1, 'Draw', 4.0, '2025-05-11T11:11:27.929119', '{"id": 117314740795, "fixture_id": 19138512, "market_id": 1, "bookmaker_id": 38, "label": "Draw", "value": "4.00", "name": "Draw", "sort_order": 2, "market_description": "Fulltime Result", "probability": "25%", "dp3": "4.000", "fractional": "4", "american": "300", "winning": true, "stopped": false, "total": null, "handicap": null, "participants": null, "create

,id,fixture_id,bookmaker_id,market_id,label,odd_value,updated_at,raw
0,117314740794,19138512,38,1,Home,6.05,2025-05-11T11:11:27.929085,"{""id"": 117314740794, ""fixture_id"": 19138512, ""..."
1,117314740795,19138512,38,1,Draw,4.00,2025-05-11T11:11:27.929119,"{""id"": 117314740795, ""fixture_id"": 19138512, ""..."
2,117314740796,19138512,38,1,Away,1.59,2025-05-11T11:11:27.929096,"{""id"": 117314740796, ""fixture_id"": 19138512, ""..."
3,117315706427,19138512,38,6,Home,1.85,2025-05-11T11:11:27.936061,"{""id"": 117315706427, ""fixture_id"": 19138512, ""..."
4,117315706428,19138512,38,6,Away,1.79,2025-05-11T11:11:27.936070,"{""id"": 117315706428, ""fixture_id"": 19138512, ""..."


we need to get the odds, as the latest updates was the 11th of may.